In [1]:
import pandas as pd
import numpy as np

In [ ]:
prices = pd.read_csv('data/msci_prices.csv')
prices['date'] = pd.to_datetime(prices['date'])
prices = prices.set_index('date')

In [82]:
returns = prices.pct_change()

C:\Users\bapdu\AppData\Local\Temp\ipykernel_30756\3873685271.py:1: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [83]:
weights = pd.read_excel('data/indices.xlsx', sheet_name='Composition MSCI World')
weights['Dates'] = pd.to_datetime(weights['Dates'])
weights = weights.set_index('Dates')

KeyError: 'Dates'

In [ ]:
aligned_prices = prices.reindex(weights.index)
aligned_returns = aligned_prices.pct_change()

C:\Users\bapdu\AppData\Local\Temp\ipykernel_30756\132733366.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  aligned_returns = aligned_prices.pct_change()


In [ ]:
common_tickers = aligned_returns.columns.intersection(weights.columns)
aligned_returns = aligned_returns[common_tickers]
weights = weights[common_tickers]

In [ ]:
# Identifier les rendements non nuls
non_null_returns_mask = aligned_returns.isna() == False

# Calculer la somme des poids pour les rendements non nuls à chaque date
non_null_weights_sum = (weights * non_null_returns_mask).sum(axis=1)
non_null_weights_sum.iloc[0] = 1


In [ ]:
weighted_returns = (aligned_returns * weights).sum(axis=1)/non_null_weights_sum
initial_index_value = 100  # Par exemple, l'indice commence à 100
index_values = (1 + weighted_returns).cumprod() * initial_index_value

# Créer un DataFrame pour l'indice recomposé
index_df = pd.DataFrame({'Index Value': index_values})

In [ ]:
index_df

,Index Value
Dates,
2007-04-30,100.000000
2007-05-31,104.751269
2007-06-29,104.039311
2007-07-31,101.112493
2007-08-31,102.143800
...,...
2024-10-31,1188.130545
2024-11-29,1287.625184
2024-12-31,1289.813086


In [67]:
import plotly.graph_objects as go

# Tracer l'évolution de l'indice
fig = go.Figure()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer une figure avec deux sous-graphiques (un pour chaque courbe)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    subplot_titles=("Index Value Over Time", "Non-Null Weights Sum Over Time"))

# Ajouter la courbe de l'indice (premier graphique)
fig.add_trace(go.Scatter(
    x=index_df.index,
    y=index_df['Index Value'],
    mode='lines',
    name='Index Value',
    line=dict(color='blue')
), row=1, col=1)

# Ajouter la courbe de la somme des poids non nuls (deuxième graphique)
fig.add_trace(go.Scatter(
    x=non_null_weights_sum.index[1:],
    y=non_null_weights_sum[1:],
    mode='lines',
    name='Non-Null Weights Sum',
    line=dict(color='orange')
), row=2, col=1)

# Ajouter des titres et personnaliser le graphique
fig.update_layout(
    title='Index Value and Non-Null Weights Sum Over Time',
    xaxis_title='Date',
    yaxis_title='Value',
    height=600,  # Ajuster la hauteur de la figure
    template='plotly_white'
)

# Afficher le graphique
fig.show()

In [ ]:
### Save to parquet 
prices = pd.read_csv('data/msci_prices.csv')
prices = prices.rename(columns={'date': 'Date'})
prices.to_parquet('data/msci_prices.parquet', index=False)

In [76]:
weights = pd.read_excel('data/indices.xlsx', sheet_name='Composition MSCI World')
weights = weights.rename(columns={'Dates': 'Date'})
weights.to_parquet('data/indices.parquet', index=False)

In [77]:
weights = pd.read_parquet('data/indices.parquet')